# Working with Partitioned Topics

In the previous notebooks, we worked with a single topic where all the information was consumed by a single consumer.

You can have several topics to make better sense of your data. What if you have similar data but you are only concerned with a portion of it. In that case you can partition parts of the data to narrow concerns.

## An example

Our resturaunt has expanded into two shops . We can use partitions to track orders between the two with our "Northern Store" being partition `0` and the "Southern Store" partition `1`.

![Produce messages to an Apache Kafka Topic partitioned](../img/produce-partition.png)

---

## Create a partitioned topic

> [!NOTE]
>
> _If you haven't already created and setup your virtual environment, follow the steps from [0-setup.ipynb](0-setup.ipynb)_

Let's create a new topic called `pizzaPartitioned` (REMEMBER topics are _case-sensitive_).

- Select the option **Do you want to enable advanced configuration?**
- Change the value of **Partitions** to `2`
- Select **Create Topic**



## Create an Apache Kafka producer

Like before let's create a Kafka producer.

In [ ]:
import os
import json

from dotenv import load_dotenv
from confluent_kafka import SerializingProducer

load_dotenv()


def json_serializer(msg, s_obj):
    return json.dumps(msg).encode('ascii')

KAFKA_SERVICE_URI = os.getenv("KAFKA_SERVICE_URI")

conf = {
    'bootstrap.servers': KAFKA_SERVICE_URI,
    'client.id': 'myclient',
    'security.protocol': 'SSL',
    'ssl.ca.location': '../sslcerts/ca.pem',
    'ssl.certificate.location': '../sslcerts/service.cert',
    'ssl.key.location': '../sslcerts/service.key', 
    'value.serializer': json_serializer,
    'key.serializer': json_serializer
    }

producer = SerializingProducer(conf)

## Push the messages to the two partitions in the `PizzaPartitioned` topic

A producer can define a partition upfront by either declaring the partition or using a custom partitioner.

In [7]:
producer.produce(
    "pizzaPartitioned",
    key={"id":1},
    value={"id":1, "name":"👨 Francesco", "pizza":"Margherita 🍕"},
    partition=0
)

producer.produce(
    "pizzaPartitioned",
    key={"id":2},
    value={"id":2, "name":"👩 Adele", "pizza":"Hawaii 🍕+🍍+🥓",  "qty": 1},
    partition=1
)

producer.flush()

0

%4|1710520613.785|FAIL|myclient#producer-4| [thrd:ssl://35.203.9.32:14181/3]: ssl://35.203.9.32:14181/3: Disconnected (after 13249916ms in state UP, 1 identical error(s) suppressed)
%4|1710521747.431|FAIL|myclient#producer-4| [thrd:ssl://34.95.2.170:14181/1]: ssl://34.95.2.170:14181/1: Disconnected (after 1133484ms in state UP, 1 identical error(s) suppressed)
%6|1710521747.462|FAIL|myclient#producer-4| [thrd:ssl://35.203.9.32:14181/3]: ssl://35.203.9.32:14181/3: Disconnected (after 1133529ms in state UP, 1 identical error(s) suppressed)
%4|1710522824.481|FAIL|myclient#producer-4| [thrd:ssl://35.203.9.32:14181/3]: ssl://35.203.9.32:14181/3: Disconnected (after 1076829ms in state UP, 1 identical error(s) suppressed)
%6|1710522824.536|FAIL|myclient#producer-4| [thrd:ssl://34.95.2.170:14181/1]: ssl://34.95.2.170:14181/1: Disconnected (after 1076953ms in state UP, 1 identical error(s) suppressed)
%6|1710522824.580|FAIL|myclient#producer-4| [thrd:ssl://kafka-22c8715b-project-348d.a.aivenclo

### Excellent!

We're going to need that last block for the remaining notebooks so let's open [consuming our partitioned topics](4-consume-partition-0.ipynb) alongside of this one.